# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.head()


In [ ]:
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Month_day'] = df['Month'].map(str) + '-' + df['Day'].map(str) 
df.drop(['Month', 'Day'], axis=1, inplace = True)
df = df[df['Month_day'] != '02-29']
df_ten_years = df[~(df['Year'] == 2015)]
df_2015 = df[df['Year'] == 2015]
df_2015.head()

In [ ]:
min_ten_years = df_ten_years.groupby('Month_day').agg({'Data_Value': 'min'})

max_ten_years = df_ten_years.groupby('Month_day').agg({'Data_Value': 'max'})

min_2015 = df_2015.groupby('Month_day').agg({'Data_Value': 'min'})
max_2015 = df_2015.groupby('Month_day').agg({'Data_Value': 'max'})

df_max = pd.merge(max_ten_years.reset_index(), max_2015.reset_index(), left_index=True, on = 'Month_day')
df_max.rename(columns={'Data_Value_x': 'Max_2005_2014', 'Data_Value_y': 'Max_2015'}, inplace=True)
df_min = pd.merge(min_ten_years.reset_index(), min_2015.reset_index(), left_index=True, on = 'Month_day')
df_min.rename(columns={'Data_Value_x': 'Min_2005_2014', 'Data_Value_y': 'Min_2015'}, inplace=True)
max_ten_years.head()

In [ ]:
diff_max = df_max[df_max['Max_2015'] > df_max['Max_2005_2014']]
diff_min = df_min[df_min['Min_2015'] < df_min['Min_2005_2014']]
diff_max.head()

In [ ]:
%matplotlib inline
plt.figure(figsize=(15,10))

plt.plot(max_ten_years.values, color = 'pink', label ='High')
plt.plot(min_ten_years.values, color = 'lightblue', label ='Low')

plt.xlabel('Days', fontsize=20)
plt.ylabel('Temperature', fontsize=20)
plt.title('High and low temperature records 2005-2014 versus 2015.', fontsize=20)

plt.scatter(diff_max.index.tolist(), diff_max['Max_2015'].values, c = 'darkred', marker='*',s = 100, label = "High in 2015")
plt.scatter(diff_min.index.tolist(), diff_min['Min_2015'].values, c = 'blue', marker='*', s = 100, label = "Low in 2015")


plt.gca().fill_between(range(len(max_ten_years)), 
                       np.array(max_ten_years.values.reshape(len(min_ten_years.values),)), 
                       np.array(min_ten_years.values.reshape(len(min_ten_years.values),)), 
                       facecolor = '#7f7f7f', 
                       alpha=0.1)

plt.legend(loc=4, fontsize=14, frameon=False)

plt.show()